# AI-Powered Breast Ultrasound Analysis for Tumor Detection

Breast cancer remains a significant global health concern, with early detection being crucial for effective treatment and improved patient outcomes. Traditional methods for interpreting breast ultrasound images can be time-consuming and subjective, often leading to variability in diagnostic accuracy. This notebook aims to leverage the advanced capabilities of **Gemini 1.5** to enhance the analysis of breast ultrasound images, facilitating more consistent and accurate interpretations.

With its impressive **2-million-token context window**, Gemini 1.5 is uniquely positioned to process extensive image data and textual information concurrently. This allows it to analyze ultrasound images alongside relevant medical literature, enabling the model to provide qualitative insights regarding the presence of tumors and their distinguishing features compared to normal tissue.

## Key Objectives

- **Purpose:** To demonstrate Gemini 1.5’s ability to interpret breast ultrasound images and identify tumor characteristics.
  
- **Scope:** Evaluate images from three categories—*benign*, *malignant*, and *normal*—using a few representative examples to guide the model's analysis.
  
- **Novelty:** Illustrate how the model's expansive context window can facilitate a deeper understanding of ultrasound imaging and improve diagnostic support for healthcare professionals.

## Setup & Installation

In this section, we will install the necessary packages and authenticate our session to connect with Google’s AI services, ensuring seamless interaction with Gemini 1.5. This setup will enable us to utilize the model's generative capabilities for effective image analysis.


In [1]:
# Import standard libraries
import os
import time
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
import cv2
import base64

# Import machine learning and image processing libraries
import tensorflow as tf

# Import Google Generative AI library for Gemini 1.5 API
import google.generativeai as genai
from google.generativeai import caching

# Kaggle secrets for API access
from kaggle_secrets import UserSecretsClient

# Setting Up Access to the Gemini 1.5 API

In this section, we will load the necessary secrets to access the AI Studio token, enabling us to authenticate and interact with the Google Generative AI API. This is a crucial step to ensure that our notebook can leverage the capabilities of the Gemini 1.5 model for analyzing breast ultrasound images effectively.

We will also verify that TensorFlow is configured to utilize GPU resources, which is essential for optimizing computation speed during image processing tasks. Additionally, we'll set a random seed for reproducibility, ensuring consistent results across different runs of the notebook.

The following code accomplishes these tasks:


In [2]:
# Load Kaggle secrets to access the AI Studio token
user_secrets = UserSecretsClient()
ai_studio_token = user_secrets.get_secret("gemini_token")

# Authenticate Google Generative AI API with the AI Studio token
genai.configure(api_key=ai_studio_token)

# Check if TensorFlow is using GPU for faster computation
print("TensorFlow is using GPU:", tf.config.list_physical_devices('GPU'))

# Set a seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Packages imported and AI Studio token configured successfully!")

TensorFlow is using GPU: []
Packages imported and AI Studio token configured successfully!


# Configuring and Loading the Gemini 1.5 Model

In this section, we define the configuration parameters for the Gemini 1.5 model. These parameters play a vital role in controlling the behavior of the model during content generation:

- **Temperature**: This parameter determines the randomness of the model's outputs. A higher temperature (e.g., 1) allows for more creative and varied responses, while a lower temperature results in more focused and deterministic outputs.
  
- **Top-p (Nucleus Sampling)**: This threshold (set at 0.95) helps maintain sampling diversity by limiting the number of probable tokens from which the model can generate the next token. It effectively balances between randomness and coherence.

- **Top-k**: By setting this to 64, we define the number of top probable tokens the model will consider for generating each token in the output, contributing to the richness of the generated content.

- **Max Output Tokens**: The model is configured to produce a maximum of 8192 tokens in its response, allowing for detailed analysis and insights.

- **Response MIME Type**: Specifying `text/plain` ensures that the output is in a plain text format, which is suitable for textual analysis.

After defining these configurations, we proceed to create and load the Gemini 1.5 model using the specified parameters. This step is crucial as it prepares the model for the subsequent analysis of breast ultrasound images.

The following code accomplishes these tasks:


In [3]:
# Define the configuration for the Gemini 1.5 model generation
generation_config = {
    "temperature": 1,           # Controls the randomness of outputs. Higher values mean more random outputs.
    "top_p": 0.95,              # Nucleus sampling threshold for sampling diversity.
    "top_k": 64,                # Number of top probable tokens considered for sampling.
    "max_output_tokens": 8192,  # Maximum number of tokens the model will output.
    "response_mime_type": "text/plain",  # MIME type for the output format.
}

# Create and load the Gemini 1.5 model using the above configuration
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config
)

print("gemini-1.5-flash model loaded successfully!")


gemini-1.5-flash model loaded successfully!


# Defining Directories for Breast Ultrasound Images

In this section, we specify the directories that contain the breast ultrasound images organized into three categories: benign, malignant, and normal. This structured approach allows us to easily access and process images from each category for analysis. 

- **Benign Directory**: This folder contains ultrasound images classified as benign, which typically indicate non-cancerous conditions.

- **Malignant Directory**: This folder holds images identified as malignant, representing potential cancerous tumors that require careful evaluation.

- **Normal Directory**: This folder includes ultrasound images that are deemed normal, serving as a baseline for comparison against other categories.

By clearly defining these directories, we streamline the process of loading and analyzing images in the subsequent steps of our analysis.


In [4]:
# Define the directories
benign_dir = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/"
malignant_dir = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/"
normal_dir = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/"

# Creating Video from Images

This section defines a function `create_video_from_images` that generates a video from a specified number of images in a directory while maintaining their **original resolution** to ensure the preservation of all visual details.

- **Function Definition**: The function accepts the following parameters:
  - **image_dir**: The directory containing the images.
  - **video_name**: The name of the output video file.
  - **num_images**: The maximum number of images to include in the video, defaulting to 10.
  - **fps**: Frames per second, which controls the playback speed of the video.

- **Video Creation Process**:
  1. **Image Selection**: The function generates a list of image file paths from the specified directory and randomly selects a subset of images to include in the video.
  2. **Resolution Setup**: The resolution of the video is determined by the dimensions of the first image in the selected subset, ensuring all frames in the video are of consistent size.
  3. **Video Writing**: The function sequentially reads each selected image in its original resolution and writes it to the video file, maintaining the original size and format of each image.

By using this function, a video is created from a set of images without compromising on the resolution, making it suitable for tasks requiring high-quality visual data.


In [5]:
# Function to create a video from images
def create_video_from_images(image_dir, video_name, num_images=10, fps=1):
    image_paths = [
        os.path.join(image_dir, f) for f in os.listdir(image_dir) 
        if f.endswith('.png') and "_mask" not in f
    ]
    random.shuffle(image_paths)
    selected_images = image_paths[:num_images]  # Select the first 'num_images'

    # Get the size of the first image to use for the video resolution
    first_image = cv2.imread(selected_images[0])
    height, width, layers = first_image.shape

    # Initialize video writer with original image size
    video_writer = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for path in selected_images:
        image = cv2.imread(path)
        video_writer.write(image)

    video_writer.release()  # Finalize the video file


# Defining Paths for Video Creation

In this section, we define the paths to the directories containing the breast ultrasound images for each class—benign, malignant, and normal. These paths will be used to create videos representing each category of images.

- **Directory Paths**:
  - **Benign Images**: 
    - Path: `'/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/'`
  - **Malignant Images**: 
    - Path: `'/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/'`
  - **Normal Images**: 
    - Path: `'/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/'`

By defining these paths, we ensure that the subsequent video creation process can access the appropriate images for analysis and classification.


In [6]:
# Define paths and create the video for each class
benign_dir = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/'
malignant_dir = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/'
normal_dir = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/'

# Creating Videos for Each Category

In this section, we generate videos for each category of breast ultrasound images—benign, malignant, and normal—without resizing the images. This ensures that the original quality and details of the images are preserved in the final videos.

- **Video Creation Process**:
  - The function `create_video_from_images` is called for each category, producing a video file for each.
  
- **Generated Video Files**:
  - **Benign Video**: `benign_video.mp4`
  - **Malignant Video**: `malignant_video.mp4`
  - **Normal Video**: `normal_video.mp4`

By executing these function calls, we create visually informative videos that can be utilized for further analysis and classification tasks.


In [7]:
# Create videos for each category without resizing
create_video_from_images(benign_dir, "benign_video.mp4", num_images=50)
create_video_from_images(malignant_dir, "malignant_video.mp4", num_images=50)
create_video_from_images(normal_dir, "normal_video.mp4", num_images=50)

# Uploading Videos to Gemini

In this section, we upload the generated videos for each category to the Gemini platform. This step allows for easy access and analysis of the videos within the Gemini framework.

- **Video Upload Process**:
  - The `genai.upload_file` function is used to upload each video file to Gemini.
  
- **Uploaded Videos**:
  - **Benign Video**: `benign_video.mp4`
  - **Malignant Video**: `malignant_video.mp4`
  - **Normal Video**: `normal_video.mp4`

By uploading these videos, we ensure that they are available for further classification and analysis tasks using the Gemini 1.5 model.


In [8]:
# Now upload the videos to Gemini
benign_video = genai.upload_file("benign_video.mp4")
malignant_video = genai.upload_file("malignant_video.mp4")
normal_video = genai.upload_file("normal_video.mp4")

# Adjusting the Prompt to Reference the Videos

This section defines the example prompt that will guide the LLM's analysis by providing context through video examples of benign, malignant, and normal breast ultrasound images.

- **Example Prompt**:
    "I have provided videos of breast ultrasound images to guide your classification. Please analyze and classify the following videos based on their characteristics:"

    - **Benign Video**: `benign_video`
      - This video contains images that typically feature well-defined borders and homogeneous echogenicity, indicating the absence of malignant characteristics.
    
    - **Malignant Video**: `malignant_video`
      - This video contains images that may display irregular borders, heterogeneous echogenicity, or the presence of hypoechoic regions, suggesting potential tumor presence.
    
    - **Normal Video**: `normal_video`
      - This video contains images demonstrating typical breast tissue architecture, with uniform echogenicity and no signs of abnormal structures.

By providing these videos, the LLM is informed of key visual patterns that it can use to classify and analyze subsequent images more accurately.


In [9]:
example_prompt = (
    "I have provided videos of breast ultrasound images to guide your classification. "
    "Please analyze and classify the following videos based on their characteristics:\n\n"
    f"Benign Video: {benign_video}\n"
    "This video contains images that typically feature well-defined, smooth borders and homogeneous echogenicity, suggesting the absence of malignant characteristics.\n\n"
    f"Malignant Video: {malignant_video}\n"
    "This video contains images that may display irregular borders, heterogeneous echogenicity, hypoechoic regions, or spiculated masses, which are potential indicators of malignancy. "
    "You may also observe posterior acoustic shadowing or calcifications, which further support the suspicion of a tumor.\n\n"
    f"Normal Video: {normal_video}\n"
    "This video contains images demonstrating typical breast tissue architecture, with uniform echogenicity, no visible masses, and no signs of architectural distortion or abnormal structures.\n"
)


# Testing with Example Prompt on Selected Images which are under Normal category

In this section, we define a process for analyzing breast ultrasound images by using the Gemini 1.5 model. The following steps outline how to select and process images from the "normal" category:

- **Folder Path Definition**:
  
  The path to the folder containing normal images is specified as follows:

  ```python
  folder_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/"


In [10]:
# Define the path to the folder containing normal images
folder_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/"

# Get a list of image files in the folder, excluding those with '_mask' in the filename
image_files = [
    os.path.join(folder_path, f) for f in os.listdir(folder_path) 
    if f.endswith('.png') and "_mask" not in f
]

# Pick 5 random images from the filtered list
selected_images = np.random.choice(image_files, 10, replace=False)

# Upload each selected image to Gemini 1.5 model and print the results
for img_path in selected_images:
    print(f"===============================")
    print(f"        Processing Image        ")
    print(f"===============================\n")
    print(f"Image Path: {img_path}\n")
    
    # Upload the image to Gemini
    myfile = genai.upload_file(img_path)
    
    # Create the detailed prompt
    detailed_prompt = (
        "Please analyze the provided breast ultrasound image in the context of the accompanying video, which consists of a series of images that illustrate different characteristics. "
        "Identify any features in the image that may indicate abnormal structures or tumors, taking into account the visual cues presented in the video. "
        "Focus on qualitative differences, including the presence of irregular borders, echotexture variations, or hypoechoic regions, particularly in comparison to the typical patterns observed in the video. "
        "Additionally, provide a medical description of any identified tumors, including their size and morphology when applicable. "
        "Use the examples provided in the video to inform your analysis and classification:\n\n" + example_prompt
    )

    # Generate content for the image using Gemini 1.5 model
    result = model.generate_content([
        myfile, "\n\n", detailed_prompt
    ])
    
    # Output the result from the LLM in a structured format
    print(f"{'=' * 50}")
    print(f"      Results for Image: {img_path}      ")
    print(f"{'-' * 50}")
    print(f"{result.text}")
    print(f"{'=' * 50}\n")
    
    # Display prompt details in a more readable format
    print("Prompt Details:")
    print(result.usage_metadata)


        Processing Image        

Image Path: /kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/normal (30).png

      Results for Image: /kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/normal (30).png      
--------------------------------------------------
The provided breast ultrasound image exhibits a complex echotexture with multiple hypoechoic regions scattered throughout.  This pattern deviates from the homogeneous echogenicity typical of benign breast tissue, as seen in the benign video. While the image lacks the clear definition of a well-demarcated mass, the heterogeneous echotexture and the presence of hypoechoic areas raise concerns about potential malignancy. These characteristics are consistent with the appearance of tumors in the malignant video, particularly those demonstrating irregular borders and heterogeneous echogenicity. 

While the image doesn't reveal a classic spiculated mass, the multiple hypoechoic regions c

# Testing with Example Prompt on Selected Images which are under Malignant category

In this section, we outline the process for analyzing breast ultrasound images that are classified as malignant. The following steps detail how to select and process these images using the Gemini 1.5 model:

- **Folder Path Definition**:

  The path to the folder containing malignant images is specified as follows:

  ```python
  folder_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/"


In [11]:
# Define the path to the folder containing malignant images
folder_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/"

# Get a list of image files in the folder, excluding those with '_mask' in the filename
image_files = [
    os.path.join(folder_path, f) for f in os.listdir(folder_path) 
    if f.endswith('.png') and "_mask" not in f
]

# Pick 5 random images from the filtered list
selected_images = np.random.choice(image_files, 5, replace=False)

# Upload each selected image to Gemini 1.5 model and print the results
for img_path in selected_images:
    print(f"===============================")
    print(f"        Processing Image        ")
    print(f"===============================\n")
    print(f"Image Path: {img_path}\n")
    
    # Upload the image to Gemini
    myfile = genai.upload_file(img_path)
    
    # Create the detailed prompt
    detailed_prompt = (
        "Please analyze the provided breast ultrasound image in the context of the accompanying video, which consists of a series of images that illustrate different characteristics. "
        "Identify any features in the image that may indicate abnormal structures or tumors, taking into account the visual cues presented in the video. "
        "Focus on qualitative differences, including the presence of irregular borders, echotexture variations, or hypoechoic regions, particularly in comparison to the typical patterns observed in the video. "
        "Additionally, provide a medical description of any identified tumors, including their size and morphology when applicable. "
        "Use the examples provided in the video to inform your analysis and classification:\n\n" + example_prompt
    )

    # Generate content for the image using Gemini 1.5 model
    result = model.generate_content([
        myfile, "\n\n", detailed_prompt
    ])
    
    # Output the result from the LLM in a structured format
    print(f"{'=' * 50}")
    print(f"      Results for Image: {img_path}      ")
    print(f"{'-' * 50}")
    print(f"{result.text}")
    print(f"{'=' * 50}\n")

    # Display prompt details in a readable format
    print("Prompt Details:")
    print(result.usage_metadata)


        Processing Image        

Image Path: /kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/malignant (32).png

      Results for Image: /kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/malignant (32).png      
--------------------------------------------------
The image shows a large, hypoechoic mass with irregular borders located in the right breast. The mass appears to be spiculated and has a heterogeneous echotexture, with areas of both increased and decreased echogenicity. These features, particularly the irregular borders and spiculated shape, are suggestive of malignancy based on the examples provided in the videos. The mass appears to be quite large, but an exact measurement cannot be determined from this image alone.

This mass bears resemblance to the malignant examples in the provided video, exhibiting characteristics like irregular borders, heterogeneous echotexture, and spiculation.  The presence of a hypoechoic 

# Dataset Citation

This analysis uses the breast ultrasound images dataset provided by Al-Dhabyani W, Gomaa M, Khaled H, and Fahmy A. The dataset can be cited as follows:

Al-Dhabyani, W., Gomaa, M., Khaled, H., & Fahmy, A. (2020). Dataset of breast ultrasound images. *Data in Brief, 28*, 104863. https://doi.org/10.1016/j.dib.2019.104863
